# Web Scraping and Introductory Data Analysis
// purpose of assintment


// summary of what we're going to do

In [ ]:
%pip install -r ./../requirements.txt

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
NUMBER_BLOCKS = 3
ETHERESCAN_URL = "https://etherscan.io/txs"

In [ ]:
class Scrip_Etheruim:
    url = ETHERESCAN_URL

    def __init__(self):
        self.driver = None
        self.df = pd.DataFrame(columns=['tnx_hash', 'method', 'block', 'date', 'from', 'to', 'value', 'tnx_fee'])

    def __del__(self):
        if self.driver:
            self.driver.quit()

    def _get_text(self, element):
        return element.find('a').get('href').split('/')[-1] 

    def _get_data(self, element):
        tnx_hash = element[1].text.strip()
        method = element[2].text.strip()
        block = element[3].text.strip()
        date = element[4].text.strip()
        _from = self._get_text(element[7])
        to = self._get_text(element[9])
        value = element[10].text.strip()
        tnx_fee = element[11].text.strip()
        data = [tnx_hash, method, block, date, _from, to, value, tnx_fee]

        return pd.Series(data, index=['tnx_hash', 'method', 'block', 'date', 'from', 'to', 'value', 'tnx_fee']), block


    def _extract_data_from_html(self, html_content):
        soup = BeautifulSoup(html_content, "html.parser")
        rows = soup.find_all("tr")
        block_number = 0
        for row in rows:
            cells = row.find_all("td")
            series, block = self._get_data(cells)
            block_number = max(block_number, int (block))
            self.df = pd.concat([self.df, pd.DataFrame([series])], ignore_index=True)

        return block_number

            
    def _extract_data_from_url(self):
        table_body = self.driver.find_element(By.CSS_SELECTOR, "tbody.align-middle.text-nowrap")
        html_content = table_body.get_attribute("outerHTML")
        block_number = self._extract_data_from_html(html_content)

        return block_number
    
    def _click_next_button(self):

        try:
            next_button = WebDriverWait(self.driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "a[aria-label='Next']"))
            )
            next_button.click()
        except Exception as e:
            print(f"Error clicking the 'Next' button: {e}")
            raise

    def extract_data(self):
        self.driver = webdriver.Chrome()
        self.driver.get(self.url)
        print("stage: 1")
        block_number = self._extract_data_from_url()
        print("stage: 2")
        new_block_number = block_number
        while (block_number - new_block_number) < NUMBER_BLOCKS:
            print(new_block_number, block_number)
            self._click_next_button()
            new_block_number = self._extract_data_from_url()
        
        return self.df
        
      





In [ ]:
scripEther = Scrip_Etheruim()
df = scripEther.extract_data()


In [ ]:
df